In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA = Path("../data")
RAW = DATA / "raw" / "cleaned_merged_seasons.csv"   # adjust name if different
INTERIM = DATA / "interim" / "cleaned_with_form.csv"
PROCESSED = DATA / "processed" / "train_2022_23.parquet"

RAW, RAW.exists()


(WindowsPath('../data/raw/cleaned_merged_seasons.csv'), True)

In [2]:
df = pd.read_csv(RAW, low_memory=False)

# Unify common variants
df = df.rename(columns={
    "season_x":"season", "Season":"season",
    "team_x":"team", "team_name":"team",
    "GW":"gw", "round":"gw", "gameweek":"gw",
    "web_name":"player", "player_name":"player", "name":"player",
    "position_txt":"position", "Pos":"position", "element_type":"position_code"
})

# If only codes exist, map to names
if "position" not in df.columns and "position_code" in df.columns:
    df["position"] = df["position_code"].map({1:"GK",2:"DEF",3:"MID",4:"FWD"})

# Deduplicate 'gw' column if it appears twice
if (df.columns == "gw").sum() > 1:
    first_idx = [i for i,c in enumerate(df.columns) if c=="gw"][0]
    keep = [c for i,c in enumerate(df.columns) if not (c=="gw" and i!=first_idx)]
    df = df[keep]

# Keep exactly ONE opponent column named 'opponent_team'
if "opponent_team" in df.columns:
    opp_col = "opponent_team"
elif "opp_team_name" in df.columns:
    df = df.rename(columns={"opp_team_name":"opponent_team"})
    opp_col = "opponent_team"
else:
    opp_col = None

opp_col


'opponent_team'

In [6]:
# Columns we don’t model (popularity/admin/logistic)
drop_cols = [c for c in [
    "selected","selected_by_percent","transfers_in","transfers_out","transfers_balance",
    "transfers_in_event","transfers_out_event",
    "element","fixture","kickoff_time","team_a_score","team_h_score",
    "position_code","id","code","team_code","opponent_team_code"
] if c in df.columns]
df = df.drop(columns=drop_cols)

# Ensure required columns exist
needed = ["player","season","gw","position","team","total_points","minutes"]
if opp_col: needed += [opp_col]
if "was_home" in df.columns: pass
else:
    # if you don’t have was_home, derive a placeholder False (0) column
    df["was_home"] = 0
needed += ["was_home"]
missing = [c for c in needed if c not in df.columns]
assert not missing, f"Missing required columns: {missing}"

# Basic cleanup
df["season"] = df["season"].astype(str).str.strip()
df["player"] = df["player"].astype(str).str.strip()
df["team"]   = df["team"].astype(str).str.strip()
df["position"] = (df["position"].astype(str).str.upper()
                  .str.replace("GOALKEEPER","GK").str.replace("DEFENDER","DEF")
                  .str.replace("MIDFIELDER","MID").str.replace("FORWARD","FWD"))
# --- AFTER df.rename(...), BEFORE any use of df["gw"] ---

# 1) Make column names unique (keep the first copy)
if df.columns.duplicated().any():
    # (optional) print duplicates to see what was collapsed
    # print("Duplicate cols:", df.columns[df.columns.duplicated()].tolist())
    df = df.loc[:, ~df.columns.duplicated(keep='first')]

# 2) Safety: if 'gw' still resolves to a 2-D DataFrame (rare), keep its first subcolumn
if "gw" in df.columns and isinstance(df["gw"], pd.DataFrame):
    df["gw"] = df["gw"].iloc[:, 0]

df["gw"] = pd.to_numeric(df["gw"], errors="coerce").astype("Int64")

# Coerce numerics
num_like = [c for c in df.columns if c not in ["season","player","team","position","opponent_team","was_home"]]
for c in num_like:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Fill stat NaNs with 0
fill_zero = [c for c in [
    "minutes","goals_scored","assists","clean_sheets","goals_conceded","saves",
    "own_goals","penalties_missed","penalties_saved","yellow_cards","red_cards",
    "bonus","bps","ict_index","influence","creativity","threat","value","total_points"
] if c in df.columns]
df[fill_zero] = df[fill_zero].fillna(0)

# Drop exact duplicates
df = df.drop_duplicates()

# If multiple rows per (player,season,gw) exist (double fixtures), sum stats
keys = ["player","season","gw"]
agg_map = {c:"sum" for c in fill_zero if c not in keys}
for k in ["team","position","opponent_team","was_home"]:
    if k in df.columns: agg_map[k] = "first"
df = df.groupby(keys, as_index=False).agg(agg_map).sort_values(keys).reset_index(drop=True)

df.head(1000)


,player,season,gw,minutes,goals_scored,assists,clean_sheets,goals_conceded,saves,own_goals,...,ict_index,influence,creativity,threat,value,total_points,team,position,opponent_team,was_home
0,Aaron Connolly,2020-21,1,45,0,0,0,2,0,0,...,3.4,1.2,0.3,32.0,55,1,Brighton,FWD,5,True
1,Aaron Connolly,2020-21,2,89,1,0,1,0,0,0,...,6.9,34.8,11.3,23.0,55,8,Brighton,FWD,14,False
2,Aaron Connolly,2020-21,3,73,0,0,0,2,0,0,...,1.9,0.0,12.1,8.0,55,2,Brighton,FWD,13,True
3,Aaron Connolly,2020-21,4,65,0,0,0,3,0,0,...,0.7,3.0,0.3,4.0,55,2,Brighton,FWD,7,False
4,Aaron Connolly,2020-21,5,12,0,1,0,0,0,0,...,3.0,17.2,10.3,2.0,55,4,Brighton,FWD,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Abdoulaye Doucouré,2021-22,26,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,53,0,Everton,MID,16,False
996,Abdoulaye Doucouré,2021-22,27,90,0,0,0,1,0,0,...,3.5,16.8,14.1,4.0,53,2,Everton,MID,12,True
997,Abdoulaye Doucouré,2021-22,28,90,0,0,0,5,0,0,...,4.8,17.0,26.9,4.0,53,2,Everton,MID,17,False
998,Abdoulaye Doucouré,2021-22,29,180,0,0,1,1,0,0,...,2.4,17.4,3.0,3.0,106,5,Everton,MID,20,True


In [8]:
df = df.sort_values(["player","season","gw"])
# form uses previous weeks only (no leakage)
df["form"] = (df.groupby(["player","season"])["total_points"]
                .apply(lambda s: s.shift(1).rolling(4, min_periods=1).mean()/10)
             ).reset_index(level=[0,1], drop=True)

# lags of total_points (1..4)
for k in [1,2,3,4]:
    df[f"lag{k}_total_points"] = df.groupby(["player","season"])["total_points"].shift(k)

df.head(50000)


,player,season,gw,minutes,goals_scored,assists,clean_sheets,goals_conceded,saves,own_goals,...,total_points,team,position,opponent_team,was_home,form,lag1_total_points,lag2_total_points,lag3_total_points,lag4_total_points
0,Aaron Connolly,2020-21,1,45,0,0,0,2,0,0,...,1,Brighton,FWD,5,True,NaN,NaN,NaN,NaN,NaN
1,Aaron Connolly,2020-21,2,89,1,0,1,0,0,0,...,8,Brighton,FWD,14,False,0.100000,1.0,NaN,NaN,NaN
2,Aaron Connolly,2020-21,3,73,0,0,0,2,0,0,...,2,Brighton,FWD,13,True,0.450000,8.0,1.0,NaN,NaN
3,Aaron Connolly,2020-21,4,65,0,0,0,3,0,0,...,2,Brighton,FWD,7,False,0.366667,2.0,8.0,1.0,NaN
4,Aaron Connolly,2020-21,5,12,0,1,0,0,0,0,...,4,Brighton,FWD,6,False,0.325000,2.0,2.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Kieran Gibbs,2017-18,11,90,0,0,0,1,0,0,...,2,nan,DEF,8,False,0.150000,1.0,2.0,2.0,1.0
49996,Kieran Gibbs,2017-18,12,90,0,0,0,4,0,0,...,0,nan,DEF,5,True,0.175000,2.0,1.0,2.0,2.0
49997,Kieran Gibbs,2017-18,13,90,0,0,0,1,0,0,...,1,nan,DEF,17,False,0.125000,0.0,2.0,1.0,2.0
49998,Kieran Gibbs,2017-18,14,79,0,1,0,1,0,0,...,5,nan,DEF,13,True,0.100000,1.0,0.0,2.0,1.0
